### RAG with PDF Data extraction to give context to LLM

In [1]:
!pip install pypdf

In [1]:
from dotenv import load_dotenv
load = load_dotenv('./../.env')


In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:1b",
    temperature=0.5,
    max_tokens=250
)

### 1. Extracting the PDF files

In [3]:
from langchain_community.document_loaders import PyPDFLoader

pdf1 = "./attention.pdf"
pdf2 = './LLMForgetting.pdf'
pdf3 = "./TestingAndEvaluatingLLM.pdf"
pdf4 = './hieu-ve-trai-tim.pdf'
pdf5 = "./zlib.pub_unlocking-the-customer-value-chain.pdf"

pdfFiles = [pdf5]

documents = []

for pdf in pdfFiles:
    loader = PyPDFLoader(pdf)
    documents.extend(loader.load())

print(len(documents))

316


In [4]:
print(documents[:1])

[Document(metadata={'producer': 'calibre 2.53.0 [http://calibre-ebook.com]', 'creator': 'calibre 2.53.0 [http://calibre-ebook.com]', 'creationdate': '2017-04-14T13:28:12+00:00', 'author': 'Minh Niệm', 'title': 'Hiểu Về Trái Tim', 'source': './hieu-ve-trai-tim.pdf', 'total_pages': 279, 'page': 0, 'page_label': '1'}, page_content='')]


### Text Splitting

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_splits = text_splitter.split_documents(documents)

len(all_splits)

842

### 3. Embedding

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
len(vector_1), len(vector_2)

(768, 768)

In [18]:
!pip install -qU langchain-chroma

^C


In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [1]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 2}
)

# retriever.batch([
#     "Mục lục là gì"
# ])

retriever.get_relevant_documents("Mục lục cuốn sách là gì")

NameError: name 'vector_store' is not defined

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

query = "Mục lục"

retrieved_docs = retriever.get_relevant_documents(query)

context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt_template = ChatPromptTemplate.from_template(
    """
    You are an AI Assisant. Use the following context to answer the question correctly.
    If you dont know the answer, just tell, I dont know.
    
    Also, summarize the response in MD format
    
    "context: {context} \n\n"
    "question: {question} \n\n"
    "AI answer:
    
    """
)

chain = prompt_template | llm | StrOutputParser()

response = chain.invoke({"context": context_text, "question": query})

print(response)

## Mục lục

**1. Lời mở đầu:**

*   Mong muốn tìm một người đặc biệt đồng hành trong cuộc đời.

**2. Về Lười biếng:**

*   Lười biếng không phải là phiền não lớn, nhưng là trở lực đáng sợ.
*   Lười biếng cản trở việc thực hiện ước mơ và sống sâu sắc.
*   Lười biếng kích động cảm xúc và dẫn đến những cơn mê bất ngờ.
*   Vượt qua lười biếng là bước vào vương quốc của thành công.

**3. Lối cũ và sự chán chường:**

*   Nhìn lại lối cũ, chưa thoát khỏi rừng mê.
*   Cảm thấy xa cách, nặng nề vì bước chân.
*   Bận tâm nắm giữ, không thể đạt được mục tiêu.
*   Mong muốn trở về như mây trắng, nhẹ nhàng trôi.

**4. Kết luận:**

*   Nỗ lực vượt qua lười biếng để luôn hăng hái là con đường dẫn đến thành công.

**MD Format:**

```md
## Mục lục

**1. Lời mở đầu:**

*   Mong muốn tìm một người đặc biệt đồng hành trong cuộc đời.

**2. Về Lười biếng:**

*   Lười biếng không phải là phiền não lớn, nhưng là trở lực đáng sợ.
*   Lười biếng cản trở việc thực hiện ước mơ và sống sâu sắc.
*   Lười biếng kích

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, return_source_documents=True)

question = "Nói về tác giả Minh Niệm"

response = qa_chain.invoke(question)

sources = set(doc.metadata.get("source", "Unknown") for doc in response["source_documents"])

print(response['result'])
print("\n📕 Sources Used:")
for source in sources:
    print(f"- {source}")

Tôi không có thông tin về tác giả Minh Niệm trong đoạn văn bạn cung cấp. Đoạn văn này chỉ nói về một tác phẩm nào đó được tác giả làm mới.

📕 Sources Used:
- ./hieu-ve-trai-tim.pdf


In [28]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="bge-m3")


vector_store = Chroma(
    persist_directory="./chroma_langchain_db",
    embedding_function=embeddings,
)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 2}
)

docs = retriever.get_relevant_documents("chánh niệm")

for d in docs:
    text = d.page_content.replace("\t", " ").replace("\n", " ")
    text = " ".join(text.split())  # gộp nhiều khoảng trắng thành 1
    print(f"Trang {d.metadata.get('page_label', d.metadata.get('page'))}:")
    print(text)
    print("-" * 50)

Trang 227:
cái tã bị dơ để kịp thời giúp đỡ. Bà mẹ quan sát tinh tường ấy chính là chánh niệm (niệm tâm). Chánh niệm là "khắc tinh" của mọi phiền não, bởi dưới ánh sáng mạnh mẽ của chánh niệm thì mọi phiền não đều bị cô lập và tan chảy. Thế mới nói chánh niệm chính là trái tim của thiền tập. Chỉ cần phát triển chánh niệm cho vững vàng thì phiền não sẽ tự diệt mà ta không cần phải cố gắng diệt trừ nó. Phiền não vốn là hiện tượng nên cảm xúc cũng là hiện tượng. Chúng được sinh ra từ sự vận hành sai lệch của cơ chế tâm lý, mà nguyên nhân chính là do nhận thức sai lầm về bản ngã. Hãy đừng quên, ta không thể dùng ý chí đàn áp nhận thức này để thay vào bằng nhận thức khác. Điều nên làm là cần siêng năng duy trì thói quen quan sát mọi diễn biến trong tâm ở mọi tình huống thì ta sẽ nhìn ra
--------------------------------------------------
Trang 227:
nó tan biến đi và nguyên nhân nào khiến nó tan biến. Quan trọng nhất không phải là triệt tiêu cho được phiền não, mà ta cần thấu hiểu cơ cấu của 

In [ ]:
from langchain_chroma import Chroma 
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.callbacks.base import BaseCallbackHandler
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Bạn là trợ lý tìm kiếm.\n"
        "Người dùng hỏi: {question}\n"
        "Hãy sinh ra 2 câu hỏi khác nhưng phải gắn chặt với câu hỏi, "
        "không được mở rộng sang mục lục hay các chương khác.\n"
        "Output từng câu một, mỗi câu trên một dòng."
    )
)

# ===== Callback để in prompt (query phụ) =====
class PrintPromptCallback(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print("=== Prompt gửi vào LLM (dùng để sinh query phụ) ===")
        for p in prompts:
            print(p)
        print("=============================================")

# ===== Embeddings + Vector Store =====
embeddings = OllamaEmbeddings(model="bge-m3") 

vector_store = Chroma( 
    persist_directory="./chroma_langchain_db", 
    embedding_function=embeddings, 
) 

retriever = vector_store.as_retriever( 
    search_type="similarity", 
    search_kwargs= {"k": 2} 
)

# ===== Setup LLM + MultiQueryRetriever =====
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:1b",
    temperature=0.5,
    max_tokens=250,
    callbacks=[PrintPromptCallback()]  # 👈 thêm callback để log query phụ
)

multi_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
    include_original=True,
)

# ===== Test với query =====
query = "So sánh công bằng với sòng phẳng"

# ----- Standard Retriever -----
print("=== STANDARD RETRIEVER ===")
docs = retriever.get_relevant_documents(query) 

for i, d in enumerate(docs, 1): 
    text = d.page_content.replace("\t", " ").replace("\n", " ") 
    text = " ".join(text.split())  # gộp nhiều khoảng trắng
    print(f"Doc {i} - Trang {d.metadata.get('page_label', d.metadata.get('page'))}: {text}")
    print("-" * 50)

# ----- MultiQueryRetriever -----
print("\n=== MULTI QUERY RETRIEVER ===")
multi_docs = multi_retriever.get_relevant_documents(query)

for i, d in enumerate(multi_docs, 1): 
    text = d.page_content.replace("\t", " ").replace("\n", " ") 
    text = " ".join(text.split())  # gộp nhiều khoảng trắng
    print(f"Doc {i} - Trang {d.metadata.get('page_label', d.metadata.get('page'))}: {text}")
    print("-" * 50)

print(f"\nStandard: {len(docs)} docs | MultiQuery: {len(multi_docs)} docs")


=== STANDARD RETRIEVER ===
Doc 1 - Trang 47: đắp. Bởi họ không có chủ trương trao đổi công bằng ngay từ buổi đầu, nói chi là sòng phẳng. Ý niệm sòng phẳng thường dễ bị nhầm lẫn với công bằng. Anh được một và tôi cũng được một, hay anh cho tôi hai thì tôi cho anh hai, đó là công bằng. Nhưng còn tùy vào mỗi xã hội và thời đại mà quy luật công bằng sẽ được thể hiện khác nhau. Sự công bằng thường được quy định trên mức cảm xúc. Cho nên, có khi người ta tự quy định mức công bằng nếu hai bên tự thỏa thuận trị giá tương xứng giữa vật trao đổi, mà không cần tuân theo quy ước chung của cộng đồng. Thí dụ, một trái bí đao có thể đổi với hai trái mướp đắng; một chuyến đò ngang có thể đổi với sáu câu vọng cổ; một bức tranh có thể đổi lấy mười bầu rượu; một lời hứa chân tình có thể đổi lấy ba trăm sáu mươi lăm ngày chờ đợi. Tuy sự trao đổi ấy được coi là công bằng, nhưng đôi bên đều ngầm hiểu rằng người kia vì cảm tình mới chấp nhận trao đổi như vậy, nên khi nào có cơ hội thì mình sẽ bù đắp
--------